<a href="https://colab.research.google.com/github/AOdrio/Datathon/blob/main/Segundo%20intento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from matplotlib import pyplot as plt
import seaborn as sns 

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression as LR
from sklearn.neighbors import KNeighborsRegressor as KNN
from sklearn.preprocessing import QuantileTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

import warnings
warnings.filterwarnings('ignore')

SEED = 42

In [2]:
url='https://drive.google.com/file/d/1KH1fB83JDAWcMRCHKFf9SmcGH7MQkupE/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_original = pd.read_csv(url)

In [ ]:
df_original.describe()

In [ ]:
ax = df_original.hist(bins=20, figsize=(15, 10))

se remplazan los null por la media de la columna

In [3]:
edad_media = df_original['edad'].mean()
edad_media

45.02159532392986

In [4]:
def transfromar_DataFrame (df):
  df = df.copy()
  df = df.drop(df.columns[0],axis=1)
  #Remplazar nulls de la edad con la media de la columna
  df['edad'] = df['edad'].fillna(edad_media, inplace=False)

  #Transformar categóricas en numéricas
  df['peorcalificacionBCU'] = (df['peorcalificacionBCU'].
                              str.replace('5','6').
                              str.replace('4','5').
                              str.replace('3','4').
                              str.replace('2B','3').
                              str.replace('2A','2').
                              str.replace('1C','1').
                              str.replace('1A','0'))
  df['peorcalificacionBCU'] = pd.to_numeric(df['peorcalificacionBCU'], downcast='integer')
  return df

In [5]:
df = transfromar_DataFrame(df_original)


In [ ]:
df.head()

In [6]:
y = df.pop("ingreso")
X = df.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)


In [7]:
# La linea base es el mediana de los ingresos
baseline_preds = pd.DataFrame(y_train.copy())
mediana = y_train.median()
baseline_preds['mediana'] = mediana
# Baseline errors, and display average baseline error
baseline_MAPE = MAPE(y_train,baseline_preds['mediana'])
print('MAPE linea base: ', round(baseline_MAPE *100, 2))
print('Mediana de ingreso: ', round(mediana))

MAPE linea base:  42.18
Mediana de ingreso:  53665


In [8]:
model_xgb = xgb.XGBRegressor(
            objective = "reg:squarederror",
            eval_metric = "mae",
            tree_method = "gpu_hist",
            booster = "gbtree",
            random_state = SEED,
            n_estimators = 500,
            learning_rate = 0.01,
            max_depth = 11,
            min_child_weight = 7,
            subsample = 0.9,
            colsample_bytree = 1,
            colsample_bynode = 0.7,
            colsample_bylevel = 0.8,
            verbosity = 0,
            n_jobs = -1,
            )

In [9]:
model_xgb.fit(X_train,y_train)
preds_xgb = model_xgb.predict(X_test)
MAPE_xgb = MAPE(y_test, preds_xgb)
print('MAPE_xgb:', round(100*MAPE_xgb, 2))

MAPE_xgb: 12.49


In [11]:
#data de test
url='https://drive.google.com/file/d/1sMkorQLOmf2cNGRwXkQLKeciGSUpL_ez/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_test_original = pd.read_csv(url)

#df_test_original.head()

In [12]:
df_test = transfromar_DataFrame(df_test_original)

In [13]:
preds_datathon = model_xgb.predict(df_test)

In [14]:
pd.DataFrame(preds_datathon).to_csv('predicciones2.csv') 